In [ ]:
## Mount Drive to colab.
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install seqeval
!pip install swish
!pip install transformers==3.0.0
!pip install tokenizers==0.10.0

# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification,BertConfig
from tokenizers import Tokenizer, BertWordPieceTokenizer 
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertForTokenClassification, AdamW
from transformers.modeling_bert import load_tf_weights_in_bert
import transformers
import logging
import argparse
import glob
import logging
import os
import random
import pdb
import json
import matplotlib.pyplot as plt
import numpy as np
import torch
from seqeval.metrics import precision_score, recall_score, f1_score , accuracy_score
logger = logging.getLogger(__name__)
from transformers import get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, accuracy_score
from seqeval.metrics import classification_report
import os
from sklearn.metrics import confusion_matrix as cnf_mtx
import sklearn.metrics


In [ ]:
class BERT(nn.Module):

    def __init__(self,model_type,model_ckpt,bert_config):
        super(BERT, self).__init__()

        # options_name = "bert-base-uncased"
        if(model_type=='tensorflow'):
          self.encoder=load_BFTC_from_TF_ckpt(bert_config, model_ckpt, BertForSequenceClassification)
        elif(model_type=='pytorch'):
          self.encoder = BertForSequenceClassification.from_pretrained(model_ckpt, config=bert_config)

        # self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']
def load_BFTC_from_TF_ckpt(bert_config, ckpt_path, model_class):
    config=bert_config
    model = transformers.BertForPreTraining(config)
    load_tf_weights_in_bert(model,config, ckpt_path)
    state_dict=model.state_dict()
    # logging.info(json.dumps(config))
    model = model_class(config)
    # Load from a PyTorch state_dict
    old_keys = []
    new_keys = []
    for key in state_dict.keys():
        new_key = None
        if 'gamma' in key:
            new_key = key.replace('gamma', 'weight')
        if 'beta' in key:
            new_key = key.replace('beta', 'bias')
        if new_key:
            old_keys.append(key)
            new_keys.append(new_key)
    for old_key, new_key in zip(old_keys, new_keys):
        state_dict[new_key] = state_dict.pop(old_key)

    missing_keys = []
    unexpected_keys = []
    error_msgs = []
    # copy state_dict so _load_from_state_dict can modify it
    metadata = getattr(state_dict, '_metadata', None)
    state_dict = state_dict.copy()
    if metadata is not None:
        state_dict._metadata = metadata

    def load(module, prefix=''):
        local_metadata = {} if metadata is None else metadata.get(prefix[:-1], {})
        module._load_from_state_dict(
            state_dict, prefix, local_metadata, True, missing_keys, unexpected_keys, error_msgs)
        for name, child in module._modules.items():
            if child is not None:
                load(child, prefix + name + '.')
    start_prefix = ''
    if not hasattr(model, 'bert') and any(s.startswith('bert.') for s in state_dict.keys()):
        start_prefix = 'bert.'
    load(model, prefix=start_prefix)
    if len(missing_keys) > 0:
        logger.info("Weights of {} not initialized from pretrained model: {}".format(
            model.__class__.__name__, missing_keys))
    if len(unexpected_keys) > 0:
        logger.info("Weights from pretrained model not used in {}: {}".format(
            model.__class__.__name__, unexpected_keys))
    if len(error_msgs) > 0:
        raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
                           model.__class__.__name__, "\n\t".join(error_msgs)))
    return model


In [ ]:

class dataset(object):
  def __init__(self, path,lang,data_type,device,batch_size_taken,fields):
      self.path=path
      self.lang=lang

      if(data_type=='train'):
        self.tabular_data,_,_=TabularDataset.splits(path=path+"/", train=data_type+'-'+lang+'.tsv',validation='valid'+'-'+lang+'.tsv',test='test'+'-'+lang+'.tsv', format='TSV', fields=fields, skip_header=True)
        self.iter=BucketIterator(self.tabular_data, batch_size=batch_size_taken, sort_key=lambda x: len(x.text), device=device, train=True, sort=False, sort_within_batch=False)
      elif(data_type=='valid'):
        # _,self.tabular_data,_=TabularDataset.splits(path=path+"/", validation=data_type+'-'+lang+'.tsv',train=data_type+'-'+lang+'.tsv',test=data_type+'-'+lang+'.tsv' format='TSV', fields=fields, skip_header=True)
        self.tabular_data=TabularDataset(path=path+"/"+data_type+'-'+lang+'.tsv', format='TSV', fields=fields, skip_header=True)
        self.iter=BucketIterator(self.tabular_data, batch_size=batch_size_taken, sort_key=lambda x: len(x.text), device=device, train=False, sort=False, sort_within_batch=False)
      else:
        # self.tabular_data=TabularDataset.splits(path=path+"/", test=data_type+'-'+lang+'.tsv', format='TSV', fields=fields, skip_header=True)
        self.tabular_data=TabularDataset(path=path+"/"+data_type+'-'+lang+'.tsv', format='TSV', fields=fields, skip_header=True)
        self.iter=Iterator(self.tabular_data, batch_size=batch_size_taken, device=device, train=False, shuffle=False, sort=False)

  def get_iter_loader(self):
  	  return self.iter
  def get_tabular_data(self):
    return self.tabular_data

def evaluate(model, test_loader,label_field,out_dir,test_lang,device,tokenizer):
    y_pred = []
    y_true = []
    text_list=[]
    model.eval()
    with torch.no_grad():
        for (labels,  text), _ in test_loader:
                # print(labels)
                labels = labels.type(torch.LongTensor)           
                labels = labels.to(device)
                text = text.type(torch.LongTensor)  
                text = text.to(device)
                loss,output = model(text, labels)
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())
                # print(len(y_pred),len(y_true))
                text_list.extend([[tokenizer.id_to_token(x) for x in t] for t in text.tolist()] )
                # print(text.tolist())
                # print([tokenizer.convert_ids_to_tokens(t,True) for t in text.tolist()])
                # break

                # text_list.extend([ ' '.join(tokenizer.convert_ids_to_tokens(t,True)) for t in text.tolist()])	
    # print('Classification Report:')
    # print(y_true)
    # print(y_pred)
    # print(len(y_true),len(y_pred),len(text_list))


    outfile=open(out_dir+'/'+'pred.csv','w')
    outfile.write('True\tPredicted\tText\n')
    real_tags=list((label_field.vocab.stoi.keys()))
    
    for i in range(len(y_true)):
    	# outfile.write(str(y_true[i])+'\t'+str(y_pred[i])+'\t'+str(text_list[i]))
    	outfile.write(real_tags[y_true[i]]+'\t'+real_tags[y_pred[i]]+'\t'+str(text_list[i])+'\n')
    outfile.close()

    # label_field.vocab.stoi.keys()
    print("Evaluation on Language: ",test_lang)
    print("Accuracy Score ",sklearn.metrics.accuracy_score(y_true,y_pred))
    print("Micro F1",sklearn.metrics.f1_score(y_true,y_pred,average='micro'))
    print("Macro F1",sklearn.metrics.f1_score(y_true,y_pred,average='macro'))
    print("Classwise precision_recall_fscore_support:\n", sklearn.metrics.precision_recall_fscore_support(y_true,y_pred))
    print("Confusion Matrix:")
    print(sklearn.metrics.confusion_matrix([real_tags[t] for t in y_true],[real_tags[t] for t in y_pred] ) )
    outfile=open(out_dir+'/'+'report.txt','w')
    print(real_tags)
    outfile.write("Accuracy Score "+str(sklearn.metrics.accuracy_score(y_true,y_pred)))
    outfile.write("\nMicro F1"+str(sklearn.metrics.f1_score(y_true,y_pred,average='micro')))
    outfile.write("\nMacro F1"+str(sklearn.metrics.f1_score(y_true,y_pred,average='macro')))
    outfile.write("\nClasswise precision_recall_fscore_support:\n")
    outfile.write(str(sklearn.metrics.precision_recall_fscore_support(y_true,y_pred)[0])+'\n')
    outfile.write(str(sklearn.metrics.precision_recall_fscore_support(y_true,y_pred)[1])+'\n')
    outfile.write(str(sklearn.metrics.precision_recall_fscore_support(y_true,y_pred)[2])+'\n')
    outfile.write(str(sklearn.metrics.precision_recall_fscore_support(y_true,y_pred)[3])+'\n')

    outfile.write("\nConfusion Matrix: \n"+str(sklearn.metrics.confusion_matrix([real_tags[t] for t in y_true],[real_tags[t] for t in y_pred] )))
    outfile.close()
  # Training Function

def train(destination_folder,
    		  dataset_path,
          bert_config_path,
          model_type,
          model_path,
          do_lower_case,
          train_lang,
          test_langs_list,
          num_epochs = 5,
          eval_every_steps = -1,
		      MAX_SEQ_LEN = 128,
          criterion = nn.CrossEntropyLoss(),
          learning_rate=2e-5,
          batch_size=32,
          best_valid_loss = float("Inf"),
          vocab_file_path = "",
          tokenizer_path = ""):


    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print("Device: ",device)    

    if tokenizer_path:
      tokenizer = Tokenizer.from_file(tokenizer_path)
    else:
      tokenizer = BertWordPieceTokenizer(vocab_file_path, lowercase=do_lower_case, strip_accents=False)

   # Model parameter
    PAD_INDEX = tokenizer.token_to_id("[PAD]")
    UNK_INDEX = tokenizer.token_to_id("[UNK]")
    CLS_INDEX = tokenizer.token_to_id("[CLS]")
    SEP_INDEX = tokenizer.token_to_id("[SEP]")

    def encode_tok(x):
      ids = tokenizer.encode(x).ids
      # print("ensure SEP")
      # print(ids.count(SEP_INDEX))
      assert(SEP_INDEX in ids)
      if ids[0] != CLS_INDEX: ids = [CLS_INDEX] + ids
      if ids[-1] != SEP_INDEX: ids = ids + [SEP_INDEX]
      return ids

    # Fields
    # label_field = Field(sequential=False,  batch_first=True, dtype=torch.float)
    label_field = Field(sequential=False,  batch_first=True,unk_token=None)
    text_field = Field(use_vocab=False, tokenize=encode_tok, lower=False, include_lengths=False, batch_first=True,
                      fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
    fields = [('label', label_field), ('text', text_field)]


    train_data_obj=dataset(	dataset_path+'/',train_lang,'train',device,batch_size,fields)
    # train_lang_train,train_lang_valid,train_lang_test=TabularDataset.splits(path=dataset_path+"/", train='train'+'-'+train_lang+'.tsv',
                                                                            # validation='valid'+'-'+train_lang+'.tsv',test='test'+'-'+train_lang+'.tsv', format='TSV', fields=fields, skip_header=True)
    label_field.build_vocab(train_data_obj.get_tabular_data(),min_freq=1)
    # label_field.build_vocab(train_lang_train,min_freq=1)
    print("Tags: ",list((label_field.vocab.stoi.keys())))
    print("Number of Tags:",len(list((label_field.vocab.stoi.keys()))))

    bert_config            = BertConfig.from_json_file(bert_config_path)
    bert_config.num_labels=len(label_field.vocab.stoi.keys())
    model = BERT(model_type,model_path,bert_config)
    model.cuda()
    print("Model successfully loaded from",model_path)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
 
    valid_data_obj=dataset(dataset_path+'/',train_lang,'valid',device,batch_size,fields)

    test_data_obj_list=[]

    for test_lang in test_langs_list:
    	test_data_obj_list.append(dataset(dataset_path+'/',test_lang,'test',device,batch_size,fields))


    
    # initialize running values    
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()

    for epoch in range(num_epochs):
        train_loader=train_data_obj.get_iter_loader()
        len_train_epoch=len(train_loader)
        print("Epoch Num:",epoch)
        print("total steps in one epoch:",len(train_loader))
        train_step_count=0
        for (labels, text), _ in train_loader:
            labels = labels.type(torch.LongTensor)           
            labels = labels.to(device)
            text = text.type(torch.LongTensor)  
            text = text.to(device)
            # print(tokenizer.convert_ids_to_tokens(text,True))
            # print(tokenizer.convert_ids_to_tokens(text[1],True))
            # print([tokenizer.convert_ids_to_tokens(t,True) for t in text.tolist()])

            output = model(text, labels)
            loss, output = output
            if(train_step_count%20==0):
              print(train_step_count,str(loss.item()))
              # print(output)
              # print([tokenizer.convert_ids_to_tokens(t,True) for t in text.tolist()])
            train_step_count+=1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if (train_step_count==len_train_epoch) or (train_step_count==eval_every_steps):
                model.eval()
                with torch.no_grad():                    
                  print("Evaluating on validation data")
                  
                  valid_loader=valid_data_obj.get_iter_loader()
                    # validation loop
                  for (labels,  text), _ in valid_loader:
                    labels = labels.type(torch.LongTensor)           
                    labels = labels.to(device)
                    text = text.type(torch.LongTensor)  
                    text = text.to(device)
                    output = model(text, labels)
                    loss, _ = output
                    
                    valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every_steps
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))
                
                # checkpoint
                try:
                  os.mkdir(destination_folder)
                except:
                  print('error while creating : ',destination_folder)
                  temp=1
                dir_name=destination_folder+'/text_classification/'
                try:
                  os.mkdir(dir_name)
                except:
                  print('error while creating : ',dir_name)
                  temp=1

                
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(destination_folder+'/text_classification/' + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(destination_folder+'/text_classification/' + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
                break

        print("Evaluation on test data")
        for test_lang_obj in test_data_obj_list:


          try:
            os.mkdir(destination_folder)
          except:
            print('error while creating : ',destination_folder)
            temp=1
          dir_name=destination_folder+'/text_classification/'
          try:
            os.mkdir(dir_name)
          except:
            print('error while creating : ',dir_name)
            temp=1
          dir_name=dir_name+'/classification_reports/'
          # dir_name='/content/drive/My Drive/btp/bert_increased_vocab/hindi_bert/hindi_and_panjabi_bert_train_on_hi-t13n-pa/'
          try:
            os.mkdir(dir_name)
          except:
            print('error while creating : ',dir_name)
            temp=1
          dir_name=dir_name+'epoch_num_'+str(epoch)+'/'
          try:
            os.mkdir(dir_name)
          except:
            print('error while creating : ',dir_name)
            temp=1
          dir_name=dir_name+test_lang_obj.lang+'/'
          try:
            os.mkdir(dir_name)
          except:
            print('error while creating : ',dir_name)
            temp=1

          evaluate(model,test_lang_obj.get_iter_loader(),label_field,dir_name,test_lang_obj.lang,device,tokenizer)        
    
    save_metrics(destination_folder + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')


In [ ]:
do_lower_case=False ## Note. Keep it False if involving any Indian Language or Scirpt of Indian Origin
MAX_SEQ_LEN = 128 ## Depends on the model architechture
batch_size=32
learning_rate=2e-5
epochs=16 ## Number of Training Loops
eval_every_steps=400 ## Number of Training Iteration in each Training Loop after which evaluation is done. 
                     ## Final model selection manually done based on Validation Loss on Train_Language.  

bert_config_path="" ## BERT Config path.              Sample : "/content/drive/My Drive/btp/bert_increased_vocab/hindi_bert/bpe_hindi_marathi_config.json"
vocab_file_path="" ## Vocab File for model.          Sample : "/content/drive/My Drive/Experiments/uncased_L-12_H-768_A-12/vocab.txt"
tokenizer_path="" ## Path to json file corresponding to the tokenizer Sample : "/content/drive/My Drive/btp/bert_increased_vocab/hindi_bert/vocabs/docsampled_hin_en_fr_mar_pan_guj_de_nl_fy_es_pt_it_multibpe_0.3.json"
model_path="" ## Model checkpoint path. Either Pytorch or Tensorflow.              
              ## Tensorflow Sample : "/content/drive/My Drive/btp/bert_increased_vocab/hindi_bert/jun_expts/MultiLang/bpe/docsampled_hin_en_fr_mar_pan_guj_de_nl_fy_es_pt_it_multibpe_0.5/model.ckpt-192000"
              ## Pytorch    Sample : "/content/drive/My Drive/Experiments/English_RelateLM_Bengali/model.pt"
model_type='' ## 'tensorflow' or 'pytorch'
dataset_path="" ## Directory containing NER data. Sample: "/content/drive/MyDrive/btp/xtreme/download/xnli/"
train_lang= ##Languages for training Sample: 'fr'
test_langs= ##Languages for testing Sample: ['fr','es','pt','it']
out_folder="" ## Directory to save the checkpoints after each epoch. Sample : "/content/drive/My Drive/btp/bert_increased_vocab/hindi_bert/jun_expts/MultiLang/bpe/docsampled_hin_en_fr_mar_pan_guj_de_nl_fy_es_pt_it_multibpe_0.5/XNLI_run1"
train(out_folder,dataset_path,bert_config_path,model_type,model_path,do_lower_case,train_lang,test_langs, epochs,eval_every_steps,MAX_SEQ_LEN, nn.CrossEntropyLoss(),learning_rate,batch_size,float("Inf"),tokenizer_path=tokenizer_path)
